In [6]:
# RGB: https://www.rapidtables.com/web/color/RGB_Color.html
import json
import os
import cv2
import numpy as np
from matplotlib import pyplot as plt
import glob
np.random.seed(251)

In [7]:
project_path = r'D:\UCBerkeley\CourseWork\202001\W251\Homework\Project'
dir_video = r'openpose_output\video_NEW3'
dir_json = r'openpose_output\json_NEW3'
dir_optical_flow = r'openpose_output\NEW4_manual_optical_flow_output_hand4_color'
dir_transfer = r'openpose_output\NEW4_image_transfer_hand4_color'

exclusion_list = ['ASL_2008_01_11_scene9-camera1_AGAIN_v2_c1_train.mov', 
                  'ASL_2008_05_12a_scene8-camera1_AGAIN_v1_c3_test.mov', 
                  'ASL_2008_08_04_scene7-camera1_AGAIN_v2_c5_train.mov', 
                  'ASL_2011_06_14_Brady_scene2-camera1_AGAIN_v3_c7_train.mov', 
                  'ASL_2008_08_04_scene45-camera1_CHAT_v1_c4_train.mov', 
                  'ASL_2008_01_11_scene81-camera1_DRESS_v1_c0_train.mov', 
                  'ASL_2008_01_18_scene23-camera1_DRESS_v1_c1_train.mov', 
                  'ASL_2008_01_18_scene24-camera1_DRESS_v1_c3_test.mov', 
                  'ASL_2008_05_12a_scene48-camera1_DRESS_v2_c6_train.mov', 
                  'ASL_2008_05_29a_scene4-camera1_DRESS_v2_c9_train.mov', 
                  'ASL_2008_08_04_scene50-camera1_DRESS_v1_c11_train.mov', 
                  'ASL_2008_08_06_scene24-camera1_DRESS_v1_c12_train.mov', 
                  'ASL_2011_06_08_Brady_scene28-camera1_DRESS_v1_c14_train.mov', 
                  'ASL_2011_06_14_Brady_scene7-camera1_DRESS_v1_c15_train.mov']

class_list = ['AGAIN', 'ALL', 'AWKWARD', 'BASEBALL', 'BEHAVIOR', 'CAN', 'CHAT', 'CHEAP', 
              'CHEAT', 'CHURCH', 'COAT', 'CONFLICT', 'COURT', 'DEPOSIT', 'DEPRESS', 
              'DOCTOR', 'DRESS', 'ENOUGH', 'NEG']

body_feature_set = [3, 4, 6, 7]
hand_feature_set = list(range(21))
hand_feature_set1 = [0, 17, 13, 9, 5, 1]
hand_feature_set2 = [0, 18, 14, 10, 6, 2]
hand_feature_set3 = [0, 19, 15, 11, 7, 3]
hand_feature_set4 = [0, 20, 16, 12, 8, 4]

# Create some random colors
color = np.random.randint(0,255,(300,3))

In [3]:
# Remove all the files in dir_optical_flow
files = glob.glob(os.path.join(project_path, dir_optical_flow, '*'))
for f in files:
    os.remove(f)
# Remove all the files in the dir_transfer folder
for i in range(len(class_list)):
    files = glob.glob(os.path.join(project_path, dir_transfer, class_list[i], '*'))
    for f in files:
        os.remove(f)


In [4]:
mov_file_lst = [f for f in os.listdir((os.path.join(project_path, dir_video)))]
for mov_file in mov_file_lst:
    if mov_file in exclusion_list:
        continue
    print(mov_file)
    json_files_lst = [f for f in os.listdir(os.path.join(project_path, dir_json)) if os.path.splitext(mov_file)[0] in f]
    if len(json_files_lst) > 0:
        print('Found the json files')
    feature_body_dict = dict()
    feature_hand_left_dict = dict()
    feature_hand_right_dict = dict()
    
    for json_f in json_files_lst:
        with open(os.path.join(project_path, dir_json, json_f)) as ff:
            json_code = json.load(ff)
        
        # This assume there is only one person
        body_raw_lst = json_code['people'][0]['pose_keypoints_2d']
        left_hand_raw_lst = json_code['people'][0]['hand_left_keypoints_2d']
        right_hand_raw_lst = json_code['people'][0]['hand_right_keypoints_2d']
        
        for i in body_feature_set:
            temp = feature_body_dict.get(i, [])
            temp.append(body_raw_lst[3*i:3*(i+1)])
            feature_body_dict[i] = temp
            
        for i in hand_feature_set4:
            temp = feature_hand_left_dict.get(i, [])
            temp.append(left_hand_raw_lst[3*i:3*(i+1)])
            feature_hand_left_dict[i] = temp
            
        for i in hand_feature_set4:
            temp = feature_hand_right_dict.get(i, [])
            temp.append(right_hand_raw_lst[3*i:3*(i+1)])
            feature_hand_right_dict[i] = temp
    
    cap = cv2.VideoCapture(os.path.join(project_path, dir_video, mov_file))
    ret, first_frame = cap.read()
    mask = np.zeros_like(first_frame)
    for (k, v) in feature_body_dict.items():
        x_0 = int(v[0][0])
        y_0 = int(v[0][1])
        for points in v[1:]:
            x_1 = int(points[0])
            y_1 = int(points[1])
            if x_0 == 0 and y_0 == 0:
                x_0 = x_1
                y_0 = y_1
            if x_1 != 0 and y_1 != 0:
                mask = cv2.line(mask, (x_0, y_0), (x_1, y_1), color[k].tolist(), 3)
                x_0 = x_1
                y_0 = y_1
                
    for (k, v) in feature_hand_left_dict.items():
        x_0 = int(v[0][0])
        y_0 = int(v[0][1])
        for points in v[1:]:
            x_1 = int(points[0])
            y_1 = int(points[1])
            if x_0 == 0 and y_0 == 0:
                x_0 = x_1
                y_0 = y_1
            if x_1 != 0 and y_1 != 0:
                mask = cv2.line(mask, (x_0, y_0), (x_1, y_1), color[k+50].tolist(), 1)
                x_0 = x_1
                y_0 = y_1
                
    for (k, v) in feature_hand_right_dict.items():
        x_0 = int(v[0][0])
        y_0 = int(v[0][1])
        for points in v[1:]:
            x_1 = int(points[0])
            y_1 = int(points[1])
            if x_0 == 0 and y_0 == 0:
                x_0 = x_1
                y_0 = y_1
            if x_1 != 0 and y_1 != 0:
                mask = cv2.line(mask, (x_0, y_0), (x_1, y_1), color[k+100].tolist(), 1)
                x_0 = x_1
                y_0 = y_1
    
    temp_segments = len(mov_file.split('_'))
    word = mov_file.split('_')[temp_segments-4]
    counter = mov_file.split('_')[temp_segments-2]
    train_test = mov_file.split('_')[temp_segments-1].split('.')[0]
    save_file = 'mask_' + word + '_' + counter + '_' + train_test
    # print('Save as', save_file)
    plt.imsave(os.path.join(project_path, dir_optical_flow, save_file + '.png'), mask)
    if 'train' in save_file:
        plt.imsave(os.path.join(project_path, dir_transfer, word, save_file + '.png'), mask)
    cap.release()
    cv2.destroyAllWindows()


ASL_2006_10_10_scene2-camera1_AGAIN_v0_c9_train.mov
Found the json files
ASL_2006_10_10_scene2-camera1_ALL_v0_c4_train.mov
Found the json files
ASL_2006_10_10_scene3-camera1_AWKWARD_v0_c4_test.mov
Found the json files
ASL_2006_10_10_scene3-camera1_BASEBALL_v1_c1_train.mov
Found the json files
ASL_2006_10_10_scene4-camera1_CAN_v0_c6_test.mov
Found the json files
ASL_2006_10_10_scene4-camera1_CHAT_v2_c8_train.mov
Found the json files
ASL_2006_10_10_scene5-camera1_CHEAP_v0_c4_train.mov
Found the json files
ASL_2006_10_10_scene5-camera1_CHURCH_v1_c4_train.mov
Found the json files
ASL_2006_10_10_scene5-camera1_COAT_v0_c4_test.mov
Found the json files
ASL_2006_10_10_scene6-camera1_CONFLICT_v0_c7_train.mov
Found the json files
ASL_2006_10_10_scene6-camera1_COURT_v0_c7_train.mov
Found the json files
ASL_2006_10_10_scene7-camera1_DEPOSIT_v0_c5_train.mov
Found the json files
ASL_2006_10_10_scene7-camera1_DEPRESS_v0_c5_train.mov
Found the json files
ASL_2006_10_10_scene7-camera1_DOCTOR_v1_c6_trai

In [5]:

len([f for f in os.listdir(os.path.join(project_path,dir_optical_flow)) if 'train' in f])

105

In [27]:
mov_file_lst = [f for f in os.listdir((os.path.join(project_path, dir_video)))]
# for mov_file in mov_file_lst:
mov_file = mov_file_lst[0]
json_files_lst = [f for f in os.listdir(os.path.join(project_path, dir_json)) if os.path.splitext(mov_file)[0] in f]
if len(json_files_lst) > 0:
    print('Found the json files')
    
feature_body_lst = []
feature_hand_left_lst = []
feature_hand_right_lst = []

for json_f in json_files_lst:
    with open(os.path.join(project_path, dir_json, json_f)) as ff:
        json_code = json.load(ff)
    
    # This assume there is only one person
    body_raw_lst = json_code['people'][0]['pose_keypoints_2d']
    left_hand_raw_lst = json_code['people'][0]['hand_left_keypoints_2d']
    right_hand_raw_lst = json_code['people'][0]['hand_right_keypoints_2d']
    
    body_raw_lst = [i for i in body_raw_lst]
    left_hand_raw_lst = [i for i in left_hand_raw_lst if i < 1 and i != 0]
    right_hand_raw_lst = [i for i in right_hand_raw_lst if i < 1 and i != 0]
    
    feature_body_lst.append(body_raw_lst)
    feature_hand_left_lst.append(left_hand_raw_lst)
    feature_hand_right_lst.append(right_hand_raw_lst)

Found the json files


In [28]:
x = np.array(feature_body_lst).mean(axis=0)
s = ("elements in the list are ["+', '.join(['%.2f']*len(x))+"]") % tuple(x)
print(s)

elements in the list are [310.70, 129.12, 0.93, 310.92, 204.03, 0.83, 248.77, 204.33, 0.72, 218.03, 300.33, 0.77, 220.86, 218.89, 0.69, 376.02, 204.32, 0.72, 396.40, 313.73, 0.75, 359.42, 256.76, 0.74, 311.13, 382.34, 0.56, 266.61, 383.06, 0.53, 230.50, 476.41, 0.11, 0.00, 0.00, 0.00, 355.58, 385.74, 0.49, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 297.35, 119.92, 0.97, 325.41, 120.12, 0.92, 278.73, 133.92, 0.89, 344.78, 135.58, 0.87, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00]


In [24]:
x = np.array(feature_hand_left_lst).mean(axis=0)
s = ("elements in the list are ["+', '.join(['%.2f']*len(x))+"]") % tuple(x)
print(s)

elements in the list are [0.05, 0.05, 0.10, 0.24, 0.32, 0.23, 0.22, 0.24, 0.30, 0.18, 0.31, 0.39, 0.52, 0.17, 0.30, 0.34, 0.35, 0.20, 0.26, 0.30, 0.35]


In [25]:
x = np.array(feature_hand_right_lst).mean(axis=0)
s = ("elements in the list are ["+', '.join(['%.2f']*len(x))+"]") % tuple(x)
print(s)

elements in the list are [0.21, 0.23, 0.19, 0.36, 0.49, 0.46, 0.47, 0.45, 0.41, 0.51, 0.48, 0.51, 0.53, 0.52, 0.49, 0.49, 0.56, 0.52, 0.53, 0.54, 0.52]


In [26]:
"elements in the list are ["+', '.join(['%.2f']*len(x))+"]"


'elements in the list are [%.2f, %.2f, %.2f, %.2f, %.2f, %.2f, %.2f, %.2f, %.2f, %.2f, %.2f, %.2f, %.2f, %.2f, %.2f, %.2f, %.2f, %.2f, %.2f, %.2f, %.2f]'

In [21]:
list(color[0])

[89, 107, 191]